In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import warnings
warnings.filterwarnings("ignore")

import acgan
import acgan.model.gmm_gan
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import twodlearn as tdl
import tensorflow as tf
import tensorflow_datasets as tfds
from functools import reduce
import matplotlib.pyplot as plt
import datetime
BATCH_SIZE = 16
#acgan.data.DATA_DIR = '/data/marinodl/tensorflow_datasets'

RESTORE = True

In [ ]:
session = tf.InteractiveSession()

In [ ]:
print(datetime.datetime.now())

In [ ]:
dataset = acgan.data.load_celeb_a_128_cropped(BATCH_SIZE)
#dataset = acgan.data.load_mnist32(BATCH_SIZE)
input_shape = dataset.output_shapes.as_list()
print('input shape: {}'.format(input_shape))

In [ ]:
import twodlearn.debug
@tdl.debug.stop_at_error
def _test():
    model = acgan.model.gmm_gan.GmmGan(
        embedding_size=64,
        embedding={'n_components': 100},
        encoder={'units':[32, 64, 64, 64], #1024
                 'kernels':3,
                 'strides':2},
        generator={'init_shape':(4, 4, 64),
                   'units': [256, 128, 64, 32, 16],
                   'outputs': 3,
                   'kernels': 3,
                   'strides': 2},
        discriminator={'units':[16, 32, 64, 128, 256], #1024
                       'kernels':3,
                       'strides':2,
                       'dropout':None}
    )
    return model
model = _test()
#model.noise_rate.init(rate=0.001)
print('output shape: {}'.format(
    model.generator.compute_output_shape(
        input_shape=[None, 100])))

In [ ]:
iter = dataset.make_one_shot_iterator()
xreal = iter.get_next()

import twodlearn.debug
@tdl.debug.stop_at_error
def _test():
    gen = model.generator_trainer(
        BATCH_SIZE,
        optimizer={'learning_rate': 0.0005, 'beta1': 0.0}
        # pyramid_loss={'scale': 1e-6},
        # regularizer={'scale': 1e-6}
        )
        #regularizer={'scale': 1e-5})
    dis = model.discriminator_trainer(
        BATCH_SIZE, xreal=xreal, 
        optimizer={'learning_rate': 0.0005, 'beta1': 0.0}
        # regularizer={'scale': 1e-6}
        )
        #regularizer={'scale': 1e-5})
    enc = model.encoder_trainer(
        BATCH_SIZE,
        optimizer={'learning_rate': 0.0005, 'beta1': 0.0}
        )
    tdl.core.variables_initializer(gen.variables).run()
    tdl.core.variables_initializer(dis.variables).run()
    tdl.core.variables_initializer(enc.variables).run()
    return gen, dis, enc
gen, dis, enc = _test()

if RESTORE:
    saver = tf.train.Saver(tdl.core.get_variables(model))
    saver.restore(session, 'tmp/GmmGan_vars_20190910:1757.ckpt')

In [ ]:
print('step {} | dis {} | gen {}'.format(dis.train_step.eval(), dis.loss.eval(), gen.loss.eval()) )

In [ ]:
for trial in tqdm.tqdm(range(100)):
#if True:
    acgan.train.run_training(dis=dis, gen=gen, n_steps=200, n_logging=10)
    for i in tqdm.tqdm(range(200)):
        session.run(enc.step['encoder'])
    for i in tqdm.tqdm(range(20)):
        session.run(enc.step['embedding'])

In [ ]:
def normalize(image):
    return ((image-image.min())/(image.max()-image.min()))
fig, ax = plt.subplots(4,4, figsize=(20, 20))
ax = np.reshape(ax, 4*4)
xsim =  session.run(gen.xsim) #dis.sim_pyramid[-1].eval()
for i in range(4*4):
    image = (xsim[i][:,:,:]+1)*0.5
    ax[i].imshow(np.squeeze(normalize(image)),
                 interpolation='nearest')
    ax[i].axis('off')
plt.savefig('gen_image.pdf')

In [ ]:
import datetime
saver = tf.train.Saver(tdl.core.get_variables(model))
saver.save(
    session, 
    'tmp/{}_vars_{}{:02d}{:02d}:{:02d}{:02d}.ckpt'.format(
        type(model).__name__,
        now.year, now.month, now.day, now.hour, now.minute))

In [ ]:
#model.embedding.components[0].loc.value().eval()

In [ ]:
fig, ax = plt.subplots(2, len(dis.real_pyramid), figsize=(20, 7))
real_pyramid_ = session.run(dis.real_pyramid)
sim_pyramid_ = session.run(dis.sim_pyramid)
for idx in range(len(dis.real_pyramid)):
    ax[0, idx].imshow(np.squeeze(normalize(real_pyramid_[idx][1, ...])),
                      interpolation='nearest')
    ax[1, idx].imshow(np.squeeze(normalize(sim_pyramid_[idx][-1, ...])),
                      interpolation='nearest')

In [ ]:
_logits = model.embedding.logits.value().eval()
np.save('logits_tmp.pkl', _logits)

In [ ]:
logits_h = tf.placeholder(dtype=tf.float32)
logits_op = model.embedding.logits.assign(logits_h)

In [ ]:
_logits = model.embedding.logits.value().eval()
logits_op.eval({logits_h: np.array([30.0 if i == 71 else 0.0 
                                    for i in range(100)])})

# plot
def normalize(image):
    return ((image-image.min())/(image.max()-image.min()))
fig, ax = plt.subplots(4,4, figsize=(15, 15))
ax = np.reshape(ax, 4*4)
xsim =  session.run(gen.xsim) #dis.sim_pyramid[-1].eval()
for i in range(4*4):
    image = (xsim[i][:,:,:]+1)*0.5
    ax[i].imshow(np.squeeze(normalize(image)),
                 interpolation='nearest')
    ax[i].axis('off')
plt.savefig('gen_image.pdf')

# restore
logits_op.eval({logits_h: _logits});

In [8]:
import twodlearn as tdl
import tensorflow.keras.layers as tf_layers
from acgan.model.msg_gan import (MSG_GAN, MSG_DiscriminatorTrainer, MSG_GeneratorTrainer,
                      AffineLayer, Conv2DLayer, LEAKY_RATE)

units = [32, 64, 64, 64]
kernels = [[3,3], [3,3], [3,3], [3,3]]
strides = [[2,2], [2,2], [2,2], [2,2]]
padding = 'same'
dropout=None
embedding_size = 64 

n_layers = len(units)
padding = (padding if isinstance(padding, (list, tuple))
           else [padding]*n_layers)

model = tdl.stacked.StackedModel()
for i in range(len(units)):
    model.add(Conv2DLayer(
        filters=units[i], strides=strides[i],
        kernel_size=kernels[i], padding=padding[i]))
    model.add(tf_layers.LeakyReLU(LEAKY_RATE))
    if dropout is not None:
        model.add(tf_layers.Dropout(rate=dropout))

model.add(tf_layers.Flatten())
model.add(AffineLayer(units=embedding_size))

In [9]:
model.add(tdl.bayesnet.NormalModel(
            loc=lambda x: x,
            batch_shape=embedding_size
        ))

In [10]:
import tensorflow as tf
y = model(tf.zeros([10, 32, 32, 3]))

In [15]:
tdl.core.array.reduce_sum_rightmost(y.log_prob(tf.zeros([10, 64])))

<tf.Tensor 'Sum:0' shape=(10,) dtype=float32>